In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy.spatial.transform.rotation import Rotation as R
import plotly.graph_objects as go
import magpylib as magpy
from magpylib._src.display.plotly.plotly_base_traces import make_BasePrism

magpy.defaults.display.backend = "plotly"


def make_wheel(Ncubes=6, height=10, diameter=36, path_len=5, name=None):
    cs_lambda = lambda: magpy.magnet.Cuboid(
        (1, 0, 0), dimension=[height] * 3, position=(diameter / 2, 0, 0)
    )
    s0 = cs_lambda().rotate_from_angax(
        np.linspace(0.0, 360.0, Ncubes, endpoint=False), "z", anchor=(0, 0, 0), start=0
    )
    cs_list = []
    for ind in range(Ncubes):
        s = cs_lambda()
        s.position = s0.position[ind]
        s.orientation = s0.orientation[ind]
        cs_list.append(s)
    c = magpy.Collection(cs_list)
    c.rotate_from_angax(90, "x")
    c.rotate_from_angax(
        np.linspace(90, 360, path_len), axis="z", start=0, anchor=(80, 0, 0)
    )
    c.move(np.linspace((0, 0, 0), (0, 0, 200), path_len), start=0)
    c.style.name = name

    trace = make_BasePrism(
        base_vertices=Ncubes, diameter=diameter + height * 2, height=height * 0.5
    )
    trace_plotly = {**trace, "opacity": 0.5, "color": "blue"}
    c.style.model3d.extra = [dict(backend="plotly", trace=trace_plotly)]
    return c


def create_compound_set(show=True, **kwargs):
    c1 = make_wheel(name="Magnetic Wheel c1 moved/rotated")
    c1.set_children_styles(
        path_show=False, 
        magnetization_color_north='magenta',
        magnetization_color_south='cyan')
    c2 = make_wheel(name="Magnetic Wheel c1")
    c2.style.model3d.extra[0].trace["color"] = "red"
    c2.style.model3d.extra[0].trace["opacity"] = 0.1
    c2.set_children_styles(path_show=False, opacity=0.1)
    #c2.move((-200, 0, 0))
    for k, v in kwargs.items():
        setattr(c1, k, eval(v))
    if show is not None:
        fig = go.Figure()
        magpy.display(c2, c1, path=1, canvas=fig)
        fig.layout.title = ', '.join(f"c1.{k} = {v}" for k,v in kwargs.items())
        fig.show()
    return c1,c2

c1, c2 = create_compound_set(show=True)
create_compound_set(show=True, orientation='None')
create_compound_set(show=True, position='np.array([[50, 0, 100]] * 2)')
create_compound_set(show=True, orientation='R.from_rotvec([[90,0,0],[0,90,0]], degrees=True)')
create_compound_set(show=True, position='np.array(np.linspace((280.,0.,0), (280.,0.,300), 8))')
create_compound_set(show=True, orientation='R.from_rotvec([[0,90*i,0] for i in range(6)], degrees=True)')